# Line flux existence check

> Check that line fluxes are not measured where they shouldn't be, and vice versa.

In [ ]:
# |default_exp diagnostics.line_flux_check

In [ ]:
# |export

import numpy as np
import xarray as xr

from qagmire.data import (
    get_lr_l2_stack_files,
    read_class_spec,
    read_class_table,
    read_galaxy_table,
)
from qagmire.quality_assurance import Diagnostics
from qagmire.utilities import parse_line_names

To write checks of the data, we create a subclass of `Diagnostics` and implement the `tests` method.

In [ ]:
# |exports


class LineFluxCheck(Diagnostics):
    """Line flux existence check.

    A reproduction of the weaveio [line_flux_check](https://github.com/bamford/QAG/blob/master/diagnostics/line_flux_check.py).

    This tests for the following cases:

    * Do non-null line fluxes appear in completely null spectra?
    * Do non-null line fluxes appear in the blue chip gap?
    * Do non-null line fluxes appear in the red chip gap?
    * Do non-null line fluxes appear outside the observed wavelength range?
    * Do null line fluxes appear in an observed wavelength range?
    """

    @staticmethod
    def _line_wavelengths(
        data: xr.Dataset,  # merged galaxy_table and class_table
    ) -> xr.Dataset:  # the observed wavelength of every potential line
        """Determine the expected observed wavelengths of all potential lines."""
        line_species, line_rest_wl = parse_line_names(data["LINE"])
        line_wl = (1 + data["Z"]) * line_rest_wl
        return line_wl

    @staticmethod
    def _wavelength_boundaries(
        data: xr.Dataset,  # including class_table, providing the rebinned spectra to check
    ) -> tuple[dict, dict]:  # the determined boundaries
        """Determine wavelength boundaries and wavelength gaps of blue and red spectra.

        Where a spectrum is entirely null, the returned gaps and boundaries will also be null.

        Returns two dictionaries, `boundaries` and `gaps`, each containing `low` and `high` entries,
        which are Datasets giving the low and high boundaries and gap edges determined for each spectrum.
        """
        gaps = {}
        boundaries = {}
        for band, low, high in (("B", 4000, 6000), ("R", 6000, 9000)):
            wl_dim = f"LAMBDA_{band}"
            wl = data[wl_dim]
            null_flux = data[f"FLUX_RR_{band}"].isnull()
            wl_null = wl.where(null_flux & (wl > low) & (wl < high))
            wl_not_null = wl.where(~null_flux)
            with np.errstate(invalid="ignore"):
                gaps[band] = {
                    "low": wl_null.min(dim=wl_dim),
                    "high": wl_null.max(dim=wl_dim),
                }
                boundaries[band] = {
                    "low": wl_not_null.min(dim=wl_dim),
                    "high": wl_not_null.max(dim=wl_dim),
                }
        return boundaries, gaps

    def tests(self, **kwargs):
        lr_l2_stack_files = get_lr_l2_stack_files(**kwargs)

        data = xr.merge(
            (
                read_class_spec(lr_l2_stack_files),
                read_galaxy_table(lr_l2_stack_files),
                read_class_table(lr_l2_stack_files),
            )
        )

        # perform the tests by OBID, rather than filename
        data = data.swap_dims(filename="OBID")

        line_wl = self._line_wavelengths(data)
        boundaries, gaps = self._wavelength_boundaries(data)

        measured_line_flux = data["LINES"].sel(QTY="FLUX", drop=True)
        null_flux = measured_line_flux.isnull()

        is_in_red_gap = (line_wl > gaps["R"]["low"]) & (line_wl < gaps["R"]["high"])
        is_in_blue_gap = (line_wl > gaps["B"]["low"]) & (line_wl < gaps["B"]["high"])

        # ignore gaps in completely null spectra
        is_in_red_gap = is_in_red_gap.fillna(False)
        is_in_blue_gap = is_in_blue_gap.fillna(False)

        is_in_gap = is_in_blue_gap | is_in_red_gap

        is_off_spectrum = (
            (line_wl < boundaries["B"]["low"]) | (line_wl > boundaries["B"]["high"])
        ) & ((line_wl < boundaries["R"]["low"]) | (line_wl > boundaries["R"]["high"]))

        is_on_spectrum = ~is_in_gap & ~is_off_spectrum

        # ignore whether on/off spectrum for completely null spectra
        is_off_spectrum = is_off_spectrum.fillna(False)
        is_on_spectrum = is_in_blue_gap.fillna(False)

        null_spectrum = (
            boundaries["B"]["low"].isnull() | boundaries["R"]["low"].isnull()
        )

        tests = [
            {
                "name": "line_in_null_spectrum",
                "description": "Do non-null line fluxes appear in completely null spectra?",
                "test": ~null_flux & null_spectrum,
            },
            {
                "name": "line_in_blue_chip_gap",
                "description": "Do non-null line fluxes appear in the blue chip gap?",
                "test": ~null_flux & is_in_blue_gap,
            },
            {
                "name": "line_in_red_chip_gap",
                "description": "Do non-null line fluxes appear in the red chip gap?",
                "test": ~null_flux & is_in_red_gap,
            },
            {
                "name": "line_off_spectrum",
                "description": "Do non-null line fluxes appear outside the observed wavelength range?",
                "test": ~null_flux & is_off_spectrum,
            },
            {
                "name": "null_line_on_spectrum",
                "description": "Do null line fluxes appear in an observed wavelength range?",
                "test": null_flux & is_on_spectrum,
            },
        ]
        return tests

In [ ]:
tests = LineFluxCheck()
tests.run(date="201*")

Locating and converting where necessary: 100%|██████████| 17/17 [00:00<00:00, 3093.95it/s]
Reading netCDF files... took 1.71 s. Size is 4851.652 Mb
Locating and converting where necessary: 100%|██████████| 17/17 [00:00<00:00, 4429.04it/s]
Reading netCDF files... took 2.94 s. Size is 77.962 Mb
Locating and converting where necessary: 100%|██████████| 17/17 [00:00<00:00, 8410.38it/s]
Reading netCDF files... took 3.16 s. Size is 509.241 Mb
Tests took 5.23 s to perform.
line_in_null_spectrum:
    Do non-null line fluxes appear in completely null spectra?
line_in_blue_chip_gap:
    Do non-null line fluxes appear in the blue chip gap?
line_in_red_chip_gap:
    Do non-null line fluxes appear in the red chip gap?
line_off_spectrum:
    Do non-null line fluxes appear outside the observed wavelength range?
null_line_on_spectrum:
    Do null line fluxes appear in an observed wavelength range?


In [ ]:
tests.summary(by="OBID", top=None, show_passed_tests=True)

failed                                             \
test line_in_null_spectrum line_in_blue_chip_gap line_in_red_chip_gap   
OBID                                                                    
3900                     0                   162                  226   
3372                     0                   144                  178   
3756                     0                   161                  194   
3653                     0                   165                  196   
3295                     0                   141                  200   
3803                     0                   164                  195   
3806                     0                   165                  202   
3802                     0                   158                  185   
3217                     0                    37                   87   
3346                     0                    22                   94   
3434                     0                    37                   63   
3175                     0                    52                   44   
3380                     0                    32                   46   
3133                     0                    10                   22   
3189                     0                    20                   26   
3170                     0                    15                   37   
3191                     0                     5                    7   

                                             total fails  
test line_off_spectrum null_line_on_spectrum              
OBID                                                      
3900              2646                     2        3036  
3372              2695                     4        3021  
3756              2651                     1        3007  
3653              2558                     1        2920  
3295              2541                     0        2882  
3803              2519                     3        2881  
3806              2497                     4        2868  
3802              2508                     5        2856  
3217              1781                     1        1906  
3346              1684                     1        1801  
3434               747                     0         847  
3175               709                     0         805  
3380               674                     0         752  
3133               288                     1         321  
3189               267                     2         315  
3170               261                     0         313  
3191                97                     0         109

In [ ]:
tests.summary(by="LINE")

failed                                         \
test            line_in_blue_chip_gap line_in_red_chip_gap line_off_spectrum   
LINE                                                                           
[ArIII]_7135.67                     0                   55              4343   
[SII2]_6730.68                      0                   53              3583   
[SII]_6716.31                       0                   51              3576   
[NII]_6583.34                       0                   47              3361   
Ha_6562.80                          0                   52              3313   
[OI]_6300.20                        0                   73              2776   
HeI_5875.60                         1                  136              1837   
HeII_3203.15                      160                    0              1159   
[NeV]_3345.81                     135                    0               969   
[NeV]_3425.81                     115                    0               871   
[NI]_5200.39                       51                  115               569   
[NI]_5197.90                       51                  118               566   
[OIII]_5006.77                     45                  182               200   
[NeIII]_3967.40                    81                  229                 0   
[ArIV]_4711.30                     84                  211                 0   
HeII_4685.74                       84                  191                 0   
[OIII]_4363.15                    136                  137                 0   
Hb_4861.32                         44                  194                 0   
[ArIV]_4740.10                     50                  158                 0   
[OII]_3726.03                     178                    0                 0   
[OII]_3728.73                     172                    0                 0   
[NeIII]_3868.69                   103                    0                 0   

                                      total fails  
test            null_line_on_spectrum              
LINE                                               
[ArIII]_7135.67                     0        4398  
[SII2]_6730.68                      0        3636  
[SII]_6716.31                       0        3627  
[NII]_6583.34                       0        3408  
Ha_6562.80                          0        3365  
[OI]_6300.20                        0        2849  
HeI_5875.60                         1        1975  
HeII_3203.15                        2        1321  
[NeV]_3345.81                       0        1104  
[NeV]_3425.81                       1         987  
[NI]_5200.39                        0         735  
[NI]_5197.90                        0         735  
[OIII]_5006.77                      0         427  
[NeIII]_3967.40                     4         314  
[ArIV]_4711.30                      1         296  
HeII_4685.74                        0         275  
[OIII]_4363.15                      0         273  
Hb_4861.32                          1         239  
[ArIV]_4740.10                      1         209  
[OII]_3726.03                       2         180  
[OII]_3728.73                       2         174  
[NeIII]_3868.69                    10         113

In [ ]:
tests.summary(by="APS_ID", top=10)

failed                                         \
test   line_in_blue_chip_gap line_in_red_chip_gap line_off_spectrum   
APS_ID                                                                
456                        0                    5                79   
766                        1                    7                74   
728                        3                    3                74   
989                        2                    8                69   
746                        1                    1                72   
615                        5                   15                53   
40                         1                    5                67   
62                         2                    6                64   
308                        4                    3                65   
273                        2                    3                67   

                             total fails  
test   null_line_on_spectrum              
APS_ID                                    
456                        0          84  
766                        0          82  
728                        0          80  
989                        0          79  
746                        0          74  
615                        0          73  
40                         0          73  
62                         0          72  
308                        0          72  
273                        0          72

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()